## Download a Pointset object and save it in CSV format

This example shows how to download a pointset object from an Evo workspace and how to construct a CSV file from the pointset components.

### Requirements

You must have a Seequent account with the Evo entitlement to use this notebook.

The following parameters must be provided:

- The client ID of your Evo application.
- The callback/redirect URL of your Evo application.

To obtain these app credentials, refer to the [Apps and tokens guide](https://developer.seequent.com/docs/guides/getting-started/apps-and-tokens) in the Seequent Developer Portal.

In [ ]:
import pandas as pd

from evo.notebooks import FeedbackWidget, ServiceManagerWidget
from evo.objects import ObjectAPIClient

cache_location = "data"

# Evo app credentials
client_id = "<your-client-id>"  # Replace with your client ID
redirect_url = "<your-redirect-url>"  # Replace with your redirect URL

manager = await ServiceManagerWidget.with_auth_code(
    discovery_url="https://discover.api.seequent.com",
    redirect_url=redirect_url,
    client_id=client_id,
    cache_location=cache_location,
).login()

### Use the Evo Python SDK to create an object client and a data client

In [ ]:
# The object client will manage your auth token and Geoscience Object API requests.
object_client = ObjectAPIClient(manager.get_environment(), manager.get_connector())

# The data client will manage saving your data as Parquet and publishing your data to Evo storage.
data_client = object_client.get_data_client(manager.cache)

### List all objects in the workspace.

In [ ]:
from prettytable import PrettyTable

all_objects = await object_client.list_all_objects()

table = PrettyTable(["Name", "Object ID"])
for index, obj in enumerate(all_objects):
    if "pointset" in obj.schema_id.sub_classification:
        table.add_row([obj.name.ljust(20), str(obj.id).ljust(40)])

if len(table.rows) == 0:
    print("No pointsets found. Publish a pointset using the 'publish-pointset' notebook.")
else:
    print("Pointsets found:")
    print(table)

Enter the `Object ID` value for the chosen pointset object in the cell below.

In [ ]:
object_id = "<your-object-id>"

Download the Parquet files and assemble the CSV file.

In [ ]:
downloaded_object = await object_client.download_object_by_id(object_id=object_id)

metadata = downloaded_object.metadata
downloaded_dict = downloaded_object.as_dict()

df = pd.DataFrame()

# Use the data client to download the coordinate data.
downloaded_data = await data_client.download_table(
    object_id=metadata.id,
    version_id=metadata.version_id,
    table_info=downloaded_dict["locations"]["coordinates"],
    fb=FeedbackWidget(f"Downloading coordinates data as '{downloaded_dict['locations']['coordinates']['data']}'"),
)

df = pd.concat([df, downloaded_data.to_pandas()], axis=1)

# Use the data client to download the attribute data and merge it with the coordinates data.
for attribute in downloaded_dict["locations"]["attributes"]:
    attribute_name = attribute["name"]
    attribute_type = attribute["attribute_type"]

    # Download the attribute data. Every attribute has a 'values' data file.
    values_data = await data_client.download_table(
        object_id=metadata.id,
        version_id=metadata.version_id,
        table_info=attribute["values"],
        fb=FeedbackWidget(f"Downloading attribute '{attribute_name}' values data as '{attribute['values']['data']}'"),
    )

    # If the attribute is a category, download the 'table' data as well.
    if attribute_type == "category":
        table_data = await data_client.download_table(
            object_id=metadata.id,
            version_id=metadata.version_id,
            table_info=attribute["table"],
            fb=FeedbackWidget(f"Downloading attribute '{attribute_name}' table data as '{attribute['table']['data']}'"),
        )

    # Assemble the dataframe.
    if attribute_type == "category":
        # Merge the values data with the table data.
        merged_data = pd.merge(
            values_data.to_pandas(), table_data.to_pandas(), left_on="data", right_on="key", how="left"
        )
        # Drop the 'data' and 'key' columns from the merged data.
        merged_data.drop(columns=["data", "key"], inplace=True)
        # Rename the 'value' column to the attribute name.
        merged_data.rename(columns={"value": attribute_name}, inplace=True)
        # Concatenate the merged data with the coordinates data.
        df = pd.concat([df, merged_data], axis=1)

    elif attribute_type == "scalar":
        data = values_data.to_pandas()
        # Rename the 'data' column to the attribute name.
        data.rename(columns={"data": attribute_name}, inplace=True)
        # Concatenate the data with the coordinates data.
        df = pd.concat([df, data], axis=1)

# Save the dataframe as a CSV file.
df.to_csv(f"{cache_location}/output.csv", index=False)

Success! You now have a new CSV file constructed from an Evo pointset object.

## Summary

In this example, we've completed the following:
* Listed all objects in an Evo workspace.
* Selected the ID of a pointset object.
* Downloaded all Parquet files associated with the pointset.
* Constructed a CSV file from the Parquet files and saved the file to disk.